In [70]:
import requests
import sys

import torch
import numpy as np
import openmeteo_requests

sys.path.append("../")
from backend.utils import get_grid, slice_bbox

In [71]:
req_json = {
        "lat_start": 43,
        "lat_end": 44,
        "lon_start": -79,
        "lon_end": -78,
        "start_time": "2025-10-19T13:00:00",
        "end_time": "2025-10-19T16:00:00",
        "variable": "2m_temperature"
    }

response = requests.post(
    "http://localhost:17200/query",
    headers={
        "Authorization": "Weruletheatmosphere"
    },
    json=req_json
)
print(response.status_code)
if response.status_code != 503:
    print(response.json().keys())

200
dict_keys(['data', 'code'])


In [72]:
def get_om(req_json, model: str = "ecmwf_aifs025_single") -> torch.Tensor:

        grid = get_grid(**dict(list(req_json.items())[:4]))
        latitudes, longitudes = grid.view(-1, 2).T
        params = {
            "latitude": latitudes.tolist(),
            "longitude": longitudes.tolist(),
            "hourly": "temperature_2m",
            "models": model,
            "start_hour": req_json["start_time"],
            "end_hour": req_json["end_time"],
        }
        print(params)

        responses = openmeteo_requests.Client().weather_api(
            "https://api.open-meteo.com/v1/forecast", params=params, method="POST"
        )

        # get output as grid of [time, lat, lon]
        output = torch.Tensor(np.stack(
            [
                np.stack(
                    [
                        r.Hourly().Variables(i).ValuesAsNumpy() 
                        for i in range(r.Hourly().VariablesLength())
                    ],
                    axis=-1
                )
                for r in responses
            ],
            axis=1
        )).reshape(-1, *grid.shape[:2])
        output += 273.15
        return output

om_data = get_om(req_json)

{'latitude': [43.0, 43.0, 43.0, 43.0, 43.0, 43.25, 43.25, 43.25, 43.25, 43.25, 43.5, 43.5, 43.5, 43.5, 43.5, 43.75, 43.75, 43.75, 43.75, 43.75, 44.0, 44.0, 44.0, 44.0, 44.0], 'longitude': [-79.0, -78.75, -78.5, -78.25, -78.0, -79.0, -78.75, -78.5, -78.25, -78.0, -79.0, -78.75, -78.5, -78.25, -78.0, -79.0, -78.75, -78.5, -78.25, -78.0, -79.0, -78.75, -78.5, -78.25, -78.0], 'hourly': 'temperature_2m', 'models': 'ecmwf_aifs025_single', 'start_hour': '2025-10-19T13:00:00', 'end_hour': '2025-10-19T16:00:00'}


In [73]:
response_data = torch.Tensor(response.json()["data"])
print(response_data.shape)
print(om_data.shape)

print("RMSE Open Meteo and Aurora", ((response_data - om_data) ** 2).mean().sqrt().item())

torch.Size([4, 5, 5])
torch.Size([4, 5, 5])
RMSE Open Meteo and Aurora 25.284103393554688


In [74]:
from backend.prediction import Prediction
import os

prediction_stored = Prediction.from_file("/root/.cache/aurora/prediction_steps49.npz")

In [ ]:

req_box = slice_bbox(
    prediction_stored.temperature[0:4],
    (lat_start, lat_end, lon_start, lon_end),
    lat_dim=1,
)

(293, 721, 1440)